# Validaciones en Pipelines

Este notebook demuestra cómo implementar validaciones efectivas en pipelines de datos.

**Referencia:** [Validaciones](../calidad/validaciones/validaciones.md)

**Objetivos:**
- Validar esquemas de datos
- Validar rangos y reglas de negocio
- Integrar validaciones en pipelines

## 1. Importar librerías

In [ ]:
import pandas as pd
import numpy as np

print("✅ Librerías importadas")

## 2. Validación de esquema

In [ ]:
def validar_esquema(df, esquema_esperado):
    """Valida que el DataFrame tenga el esquema esperado."""
    errores = []
    
    # Verificar columnas
    columnas_esperadas = set(esquema_esperado.keys())
    columnas_actuales = set(df.columns)
    
    if columnas_esperadas != columnas_actuales:
        faltantes = columnas_esperadas - columnas_actuales
        extras = columnas_actuales - columnas_esperadas
        
        if faltantes:
            errores.append(f"Columnas faltantes: {faltantes}")
        if extras:
            errores.append(f"Columnas no esperadas: {extras}")
    
    # Verificar tipos
    for col, tipo_esperado in esquema_esperado.items():
        if col in df.columns:
            tipo_actual = str(df[col].dtype)
            if tipo_actual != tipo_esperado:
                errores.append(
                    f"Columna {col}: esperado {tipo_esperado}, actual {tipo_actual}"
                )
    
    if errores:
        raise ValueError(f"Errores de esquema:\n" + "\n".join(errores))
    
    return True

print("✅ Función de validación de esquema definida")

In [ ]:
# Ejemplo de uso
df = pd.DataFrame({
    'nombre': ['Juan', 'María'],
    'edad': [28, 35],
    'precio': [10.5, 20.0]
})

esquema = {
    'nombre': 'object',
    'edad': 'int64',
    'precio': 'float64'
}

try:
    validar_esquema(df, esquema)
    print("✅ Esquema válido")
except ValueError as e:
    print(f"❌ Error: {e}")

## 3. Validación de rangos

In [ ]:
def validar_rangos(df):
    """Valida rangos de valores."""
    errores = []
    
    # Edad entre 0 y 120
    if 'edad' in df.columns:
        fuera_rango = df[(df['edad'] < 0) | (df['edad'] > 120)]
        if len(fuera_rango) > 0:
            errores.append(f"Edades fuera de rango: {len(fuera_rango)} filas")
    
    # Precio positivo
    if 'precio' in df.columns:
        negativos = df[df['precio'] < 0]
        if len(negativos) > 0:
            errores.append(f"Precios negativos: {len(negativos)} filas")
    
    if errores:
        raise ValueError("Errores de rango:\n" + "\n".join(errores))
    
    return True

print("✅ Función de validación de rangos definida")

In [ ]:
# Ejemplo con datos válidos
df_valido = pd.DataFrame({
    'edad': [25, 30, 35],
    'precio': [10.5, 20.0, 15.0]
})

try:
    validar_rangos(df_valido)
    print("✅ Rangos válidos")
except ValueError as e:
    print(f"❌ Error: {e}")

In [ ]:
# Ejemplo con datos inválidos
df_invalido = pd.DataFrame({
    'edad': [25, 150, 35],  # 150 fuera de rango
    'precio': [10.5, -5.0, 15.0]  # Precio negativo
})

try:
    validar_rangos(df_invalido)
    print("✅ Rangos válidos")
except ValueError as e:
    print(f"❌ Error detectado (esperado):")
    print(f"   {e}")

## 4. Validación de completitud

In [ ]:
def validar_completitud(df, umbral=0.95):
    """Valida completitud mínima."""
    errores = []
    
    for col in df.columns:
        completitud = (1 - df[col].isnull().sum() / len(df)) * 100
        if completitud < umbral * 100:
            errores.append(
                f"Columna {col}: completitud {completitud:.2f}% < {umbral*100}%"
            )
    
    if errores:
        raise ValueError("Errores de completitud:\n" + "\n".join(errores))
    
    return True

print("✅ Función de validación de completitud definida")

In [ ]:
# Ejemplo
df_completo = pd.DataFrame({
    'nombre': ['Juan', 'María', 'Pedro'],
    'edad': [25, 30, 35],
    'email': ['juan@email.com', 'maria@email.com', 'pedro@email.com']
})

try:
    validar_completitud(df_completo, umbral=0.95)
    print("✅ Completitud válida")
except ValueError as e:
    print(f"❌ Error: {e}")

## 5. Pipeline con validaciones

In [ ]:
def pipeline_con_validaciones(df):
    """Pipeline con validaciones integradas."""
    print("🚀 Iniciando pipeline con validaciones...")
    
    # 1. Validar esquema
    esquema = {
        'nombre': 'object',
        'edad': 'int64',
        'precio': 'float64'
    }
    print("\n📋 Validando esquema...")
    validar_esquema(df, esquema)
    print("✅ Esquema válido")
    
    # 2. Validar rangos
    print("\n📊 Validando rangos...")
    validar_rangos(df)
    print("✅ Rangos válidos")
    
    # 3. Validar completitud
    print("\n📈 Validando completitud...")
    validar_completitud(df)
    print("✅ Completitud válida")
    
    # 4. Transformar
    print("\n🔄 Transformando datos...")
    df['total'] = df['precio'] * 1.21  # IVA
    
    print("\n✅ Pipeline completado exitosamente")
    return df

print("✅ Pipeline con validaciones definido")

In [ ]:
# Ejecutar pipeline
df_ejemplo = pd.DataFrame({
    'nombre': ['Producto A', 'Producto B', 'Producto C'],
    'edad': [25, 30, 35],
    'precio': [100.0, 200.0, 150.0]
})

resultado = pipeline_con_validaciones(df_ejemplo)
print("\nResultado:")
print(resultado)

## 6. Resumen

In [ ]:
print("=" * 60)
print("TIPOS DE VALIDACIONES")
print("=" * 60)
print("\n✅ Validación de esquema: Estructura y tipos")
print("✅ Validación de rangos: Valores en rangos esperados")
print("✅ Validación de completitud: Valores nulos")
print("✅ Validación de reglas de negocio: Lógica de negocio")
print("\n" + "=" * 60)
print("\n💡 Las validaciones deben fallar rápido y claro")
print("💡 Mejor detener el pipeline que procesar datos incorrectos")
print("=" * 60)

---

**Próximo paso:** Lee [Validaciones](../calidad/validaciones/validaciones.md) para más detalles.